In [8]:
import pandas as pd
from langchain_core.documents import Document

# Load your Excel file
df = pd.read_excel("equipments.xlsx")

# Convert each row to a LangChain Document
docs = [
    Document(
        page_content=row.to_string(),  # Stringify the row
        metadata={"row_index": idx}
    )
    for idx, row in df.iterrows()
]

# Preview the documents
docs


[Document(metadata={'row_index': 0}, page_content='SN                                     1\nslno                             C1954Y1\neqID                          NOP/STC/01\neqName    Station Computer-01 DELL-T5600\nloc                            TELEMETRY\nsrvNo                                  -\nfails                                  3'),
 Document(metadata={'row_index': 1}, page_content='SN                                     2\nslno                             81954Y1\neqID                          NOP/STC/02\neqName    Station Computer-02 DELL-T5600\nloc                            TELEMETRY\nsrvNo                                  -\nfails                                  2'),
 Document(metadata={'row_index': 2}, page_content='SN                                     3\nslno                             B1954Y1\neqID                          NOP/STC/04\neqName    Station Computer-04 DELL-T5600\nloc                            TELEMETRY\nsrvNo                                  -\nfai

In [ ]:
# import os
# from dotenv import load_dotenv
# load_dotenv()

# os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")

In [9]:
## after loading the data we will transform the data
## divide the entire document into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap=200)
documents = text_splitter.split_documents(docs)
documents[:5]

[Document(metadata={'row_index': 0}, page_content='SN                                     1\nslno                             C1954Y1\neqID                          NOP/STC/01\neqName    Station Computer-01 DELL-T5600\nloc                            TELEMETRY\nsrvNo                                  -\nfails                                  3'),
 Document(metadata={'row_index': 1}, page_content='SN                                     2\nslno                             81954Y1\neqID                          NOP/STC/02\neqName    Station Computer-02 DELL-T5600\nloc                            TELEMETRY\nsrvNo                                  -\nfails                                  2'),
 Document(metadata={'row_index': 2}, page_content='SN                                     3\nslno                             B1954Y1\neqID                          NOP/STC/04\neqName    Station Computer-04 DELL-T5600\nloc                            TELEMETRY\nsrvNo                                  -\nfai

In [13]:
## we will do embeddings to convert the data in vectors and then store in vectore store
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.vectorstores import Chroma
embedding = HuggingFaceBgeEmbeddings(model_name = "all-MiniLM-L6-v2") #Fast and Lightweight
db = Chroma.from_documents(documents=docs, embedding=embedding, persist_directory="chroma_db")


/var/folders/v3/k21yfyps5v79sdcv8v4hrjth0000gn/T/ipykernel_29671/3216767368.py:4: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceBgeEmbeddings(model_name = "all-MiniLM-L6-v2") #Fast and Lightweight
/Users/shash/Documents/projects_/chatbot_for_excel/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
## Vector database
query = "What is the name of equipment with equipment ID NOP/STC/04"
result = db.similarity_search(query)
result[0].page_content

'SN                                     1\nslno                             C1954Y1\neqID                          NOP/STC/01\neqName    Station Computer-01 DELL-T5600\nloc                            TELEMETRY\nsrvNo                                  -\nfails                                  3'

In [18]:
from langchain_community.llms import Ollama
llm = Ollama(model = "llama3.2")
llm

Ollama(model='llama3.2')

In [23]:
## Design ChatPrompt Template
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(
    """You are an assistant that helps users explore and manage equipment information from an inventory dataset. 
    Answer questions about specific equipment, summarize failure counts, identify equipment by location, or list items by attributes such as type or ID.
    I want to know about [specific detail or ID]. 
    (Examples: 
    - "List all equipment in TELEMETRY with more than 1 failure."
    - "What is the location and service number of NOP/KVM/01?"
    - "Summarize all DELL-T5600 computers and their status.")
    You can refer to equipment by:
    - ID (eqID)
    - Name (eqName)
    - Location (loc)
    - Failure count (fails)
    - Service number (srvNo)
    <context>
    {context}
    </context>
    Question : {input} """)

In [25]:
## Chain Introduction
## Create Stuff Document Chain
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(llm, prompt)

In [27]:
retriever = db.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceBgeEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x30bae4e10>, search_kwargs={})

In [29]:
## Retrieval Chain
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [31]:
response = retrieval_chain.invoke({"input":"What is the location and service number of NOP/KVM/01?"})
response['answer']

'The location for the equipment NOP/KVM/01 is EAST ANT ROOM, and its corresponding service number is AC/A0003.'